In [58]:
import subprocess
import os
import time
from openai import OpenAI
import google.generativeai as genai

In [73]:
HF_BASE_URL = "https://router.huggingface.co/v1"
HF_TOKEN = os.getenv("HF_TOKEN")  # Ensure this is set in your environment
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

In [74]:
client_openai = OpenAI(api_key=OPENAI_KEY)
client_hf = OpenAI(base_url=HF_BASE_URL, api_key=HF_TOKEN)

In [120]:
import random

random.seed(42)

shared_array = [
    random.randint(-10_000, 100) if random.random() < 0.95 else random.randint(50_000, 100_000)
    for _ in range(6000)
    # for _ in range(10_000_000)
]

with open("input_array.txt", "w") as f:
    f.write(" ".join(map(str, shared_array)))


In [52]:
problem_prompt = """
Write a Python function that reads an array of integers from a file named 'input_array.txt'. From this array, compute the maximum sum of any contiguous subarray of size exactly k=5000, where you're allowed to skip (i.e., ignore) **at most one negative number** in that subarray to boost the sum.

Skipping means: you may choose to drop one negative number's contribution within the window, but only once. If all numbers are non-negative, no skip is needed.

Print only the resulting maximum sum. Do not print anything else.
"""


convert_prompt = """
Convert the following Python code to Java. Use readable syntax and ensure all logic is correctly translated.
"""

In [110]:
def generate_with_model(client, model_name, system_msg, user_prompt):
    
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

models = {
    # "hf_llama": "meta-llama/Llama-3.3-70B-Instruct:novita",
    "huggingface": "Qwen/Qwen3-Coder-480B-A35B-Instruct:novita",
     "openai": "gpt-4o"
}

In [83]:
import re

def extract_code(text, lang=None):
    pattern = rf"```{lang or ''}\n(.*?)```" if lang else r"```.*?\n(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    lines = text.splitlines()
    clean_lines = [line for line in lines if not line.strip().lower().startswith("here's")]
    return "\n".join(clean_lines).strip()


In [105]:
import subprocess
import time

def run_python(py_file, timeout=180):
    try:
        start = time.time()
        result = subprocess.run(["python", py_file], capture_output=True, text=True, timeout=timeout)
        end = time.time()
        return result.stdout.strip(), result.stderr.strip(), end - start
    except subprocess.TimeoutExpired:
        return "", "Timeout", timeout

def run_java(java_file, timeout=180):
    classname = java_file.replace(".java", "")
    try:
        subprocess.run(["javac", java_file], check=True)
        start = time.time()
        # Run the Java class file
        result = subprocess.run(["java", classname], capture_output=True, text=True, timeout=timeout)
        end = time.time()
        return result.stdout.strip(), result.stderr.strip(), end - start
    except subprocess.TimeoutExpired:
        return "", "Timeout", timeout
    except subprocess.CalledProcessError as e:
        return "", str(e), 0


In [98]:
results = {}

for name, model_id in models.items():
    print(f"\n--- Generating Python code using {name} ({model_id}) ---")
    client = client_hf
    if name == "openai":
        client = client_openai
    # elif name == "google":
    #     client = genai.GenerativeModel(model_id)
        
    py_code = generate_with_model(client, model_id, "You are a helpful Python coding assistant.", problem_prompt)
    results[name] = {"python": py_code}

    print(f"\n--- Converting to Java using {name} ({model_id}) ---")
    java_code = generate_with_model(client, model_id, "You are a Java coding assistant.", convert_prompt + py_code)
    results[name]["java"] = java_code

for model_name, data in results.items():
    py_code = extract_code(data["python"], "python")
    java_code = extract_code(data["java"], "java")

    with open(f"{model_name}_solution.py", "w") as pyf:
        pyf.write(py_code)
    with open(f"{model_name}_solution.java", "w") as jf:
        jf.write(java_code)

print("\nCode generation complete. Files saved:")
for model in results:
    print(f"- {model}_solution.py")
    print(f"- {model}_solution.java")



--- Generating Python code using openai (gpt-4o) ---

--- Converting to Java using openai (gpt-4o) ---

Code generation complete. Files saved:
- openai_solution.py
- openai_solution.java

--- OPENAI ---
[Python]  Time: 4.87s | Output: 935257
[Java]    Time: 1.49s | Output: 935257

--- HF_LLAMA ---
[Python]  Time: 60.00s | Output:  | Error: Timeout
[Java]    Time: 60.00s | Output:  | Error: Timeout

--- HF_QWEN ---
[Python]  Time: 60.00s | Output:  | Error: Timeout
[Java]    Time: 53.05s | Output: 940462


In [121]:
with open('input_array.txt', 'r') as file:
        content = file.read().strip()
        if content:
            arr = list(map(int, content.split()))
            print(len(arr))

6000


In [125]:
for model in ["openai", "huggingface"]:
    py_file = f"{model}_solution.py"
    java_file = f"{model}_solution.java"

    # Fix class name in Java file to match file name
    with open(java_file, "r") as jf:
        java_code = jf.read()

    updated_class_name = model + "_solution"
    java_code = replace_java_class_name(java_code, updated_class_name)

    # Write it back
    with open(java_file, "w") as jf:
        jf.write(java_code)

    py_out, py_err, py_time = run_python(py_file, timeout=180)
    java_out, java_err, java_time = run_java(java_file, timeout=180)


    print(f"\n--- {model.upper()} ---")

    py_error_msg = f" | Error: {py_err[:100]}" if py_err else ""
    java_error_msg = f" | Error: {java_err[:100]}" if java_err else ""

    print(f"[Python]  Time: {py_time:.2f}s | Output: {py_out[:100]}{py_error_msg}")
    print(f"[Java]    Time: {java_time:.2f}s | Output: {java_out[:100]}{java_error_msg}")


--- OPENAI ---
[Python]  Time: 0.07s | Output: -2171148
[Java]    Time: 0.19s | Output: -2171148

--- HUGGINGFACE ---
[Python]  Time: 0.10s | Output: -2164046
[Java]    Time: 0.18s | Output: -2164046
